#### This notebook calculates the precision, recall and F1 scores for the out of sample test data in the Chemical named entity recognition test dataset text.txt file. 

In [1]:
from dataset import load_dataset
import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer


In [128]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

In [21]:
model_checkpoint = "nlp-magnets/CNER_best_magbert_10_epochs_lr_3e-05_BS_16"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
df_entity=pd.read_csv('./Chemical named entity recognition test dataset entities.txt',sep='\t',header=None,).reset_index()

In [23]:
import csv
with open('./Chemical named entity recognition test dataset text.txt') as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)


In [24]:
text_dict={}

for items in d:
    text_dict[items[0]]=items[2]

In [25]:
df_entity=df_entity.rename(columns={0:"DOI",1:'NA',2:"start_index",3:"end_index",4:"entity"})

In [26]:
df_entity['start_index']=df_entity['start_index'].astype(int)
df_entity['end_index']=df_entity['end_index'].astype(int)

In [27]:
entity_dict=defaultdict(list)
for index,row in df_entity.iterrows():
    entity_dict[row["DOI"]].append({'entity':tokenizer.decode(tokenizer(row['entity'])['input_ids'][1:-1]),\
        'start_index':row['start_index'],"end_index":row['end_index']})

In [28]:
from transformers import pipeline

# Replace this with your own checkpoint
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)


In [29]:
GT=0
for ref_dict in entity_dict:
    GT+=len(entity_dict[ref_dict])

#### Here we are calculating the true positive entities between the grount truths in the txt file and the ones our CNER pipeline catches. 

In [30]:
TP=0
model_finds=0
for k,text in text_dict.items():
    entities=token_classifier(text)
    ref_dicts=entity_dict[k]
    for entity in entities:
        model_finds+=1
        for ref_dict in ref_dicts:            
            if entity['word']==ref_dict['entity'] and\
                entity['start']==ref_dict['start_index'] and entity['end']==ref_dict['end_index']:
                TP+=1
        
    
    

In [31]:
TP

290

In [32]:
model_finds

361

In [33]:
GT

411

In [34]:
Precision=TP/model_finds

In [35]:
Precision

0.8033240997229917

In [36]:
Recall=TP/GT

In [37]:
Recall

0.7055961070559611

In [38]:
2*Precision*Recall/(Precision+Recall)

0.7512953367875648